## Working with git

In [ ]:
#| default_exp git

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
from pathlib import Path
from urllib.parse import urlparse


# Load environment variables from .env file
from dotenv import load_dotenv; load_dotenv();

In [ ]:
#| export
from lovely_docs.settings import GitSource, settings
from git import Repo, InvalidGitRepositoryError, NoSuchPathError, GitCommandError, BadName, ODBError

In [ ]:
#| export

def git_progress(op_code, cur_count, max_count=None, message=''):
    if max_count:
        print(f"\r{op_code}: {cur_count}/{max_count} {message}", end='', flush=True)
    else:
        print(f"\r{op_code}: {cur_count} {message}", end='', flush=True)


def get_repo_path(repo_url: str) -> Path:
    """Extract host and path from a git repository URL.

    Args:
        repo_url: Git repository URL (e.g., 'https://github.com/sveltejs/svelte.dev')

    Returns:
        Path object representing host/path/to/repo

    Examples:
        >>> get_repo_path('https://github.com/sveltejs/svelte.dev')
        PosixPath('github.com/sveltejs/svelte.dev')
        >>> get_repo_path('git@github.com:user/repo.git')
        PosixPath('github.com/user/repo')
    """
    # Handle SSH URLs (git@host:path)
    if repo_url.startswith('git@'):
        host_path = repo_url.replace('git@', '').replace(':', '/')
        # Remove .git suffix if present
        if host_path.endswith('.git'):
            host_path = host_path[:-4]
        return Path(host_path)

    # Handle HTTPS URLs
    parsed = urlparse(repo_url)
    path = parsed.path
    # Remove leading slash and .git suffix
    if path.startswith('/'):
        path = path[1:]
    if path.endswith('.git'):
        path = path[:-4]

    return Path(parsed.netloc) / path


def clone_repo(source: GitSource) -> tuple[str, Path]:
    """Clone a git repository if it doesn't exist, or ensure an existing one is clean and at the specified commit.

    Args:
        source (GitSource): Configuration object containing the repository URL, name, and target commit.

    Returns:
        tuple[str, Path]: Commit hash and clone directory path

    Will raise exceptions on errors
    """
    repo_path = get_repo_path(source.repo)
    clone_dir = settings.git_dir / repo_path

    try:
        repo = Repo(clone_dir)
        repo.git.clean('-fdq') # Just in case
        # Fetch to ensure remote branches are up to date
        if 'origin' in repo.remotes:
            repo.remotes.origin.fetch(progress=git_progress) # type: ignore[arg-type]
    except (InvalidGitRepositoryError, NoSuchPathError):
        clone_dir.parent.mkdir(parents=True, exist_ok=True)
        repo = Repo.clone_from(source.repo, clone_dir, progress=git_progress)
        cloned = True

    # Resolve the commit, handling branch names
    try:
        commit = repo.rev_parse(source.commit)
    except ODBError:
        commit = repo.rev_parse(f"origin/{source.commit}")

    repo.git.reset('--hard', commit.hexsha)
    return commit.hexsha, clone_dir